# Create an Interface with Minimal Energy by cycling through Miller Indices

This notebook helps identify the most energetically favorable interface between a crystalline surface and a two-dimensional (2D) material. This involves exploring various orientations and terminations of the surface to construct a corresponding matching supercell, relax it and determine which configuration minimizes the interface energy.

## 1. Set input parameters
Set the maximum Miller indices for the substrate, and the settings for the interface, relaxation, and ZSL algorithm.

In [ ]:
FILES = [
    "Ni.json",
    "Gr.json"
]

MILLER_INDICES = {
    "MAX_MILLER": {
        "H": 1,
        "K": 1,
        "L": 1
    },
    "INCLUDE_NEGATIVE": False,  # Whether to got from -H, -K, -L to H, K, L
    "USE_CONVENTIONAL_CELL": True
    # If true, surfaces will be created based on the miller indices for conventional cell 
}

SETTINGS = {
    "SUBSTRATE_PARAMETERS": {
        "MATERIAL_INDEX": 0,  # the index of the material in the materials_in list
        "MILLER_INDICES": (1, 1, 1),  # the miller indices of the interfacial plane, value will be updated in the loop
        "THICKNESS": 3,  # in layers
    },
    "LAYER_PARAMETERS": {
        "MATERIAL_INDEX": 1,  # the index of the material in the materials_in list
        "MILLER_INDICES": (0, 0, 1),  # the miller indices of the 2D material, defaulted to (0, 0, 1)
        "THICKNESS": 1,  # in layers
    },
    "INTERFACE_PARAMETERS": {
        "DISTANCE_Z": 3.0,  # in Angstroms
    },
    "ZSL_PARAMETERS": {
        "MAX_AREA": 200,  # The area to consider in Angstrom^2
        "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
        "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
        "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
        "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
    },
    "RELAXATION_PARAMETERS": {
        "RELAXER": "BFGS",  # The relaxation algorithm to use
        "CALCULATOR": "ALIGNN",  # The calculator to use (EMT, ALIGNN, etc.)
        "FMAX": 0.05,  # The maximum force allowed on each atom
    }
}


## 2. Definitions
Functions definitions to create the interfaces, sort them, relax them, and calculate the effective delta energy.

In [ ]:
from IPython.display import HTML
import base64
import io
# TODO: use Made package instead
from src.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from src.utils import ase_to_poscar, pymatgen_to_ase, to_pymatgen, get_interfacial_energy, get_adhesion_energy
from ase.io import write
from ase.build import make_supercell
from ase.optimize import BFGS
from ase.calculators.emt import EMT
import matgl
from matgl.ext.ase import M3GNetCalculator
from alignn.ff.ff import AlignnAtomwiseCalculator, default_path, revised_path, wt1_path, wt01_path, \
    wt10_path

# Type hinting
from ase import Atoms
from ase.calculators.calculator import Calculator
from ase.optimize.optimize import Optimizer
from collections.abc import Sequence
from pymatgen.core import Structure

# Set up the calculators
model_path = wt1_path()
alignn_calculator = AlignnAtomwiseCalculator(path=model_path)
pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")
m3gnet_calculator = M3GNetCalculator(pot)

OPTIMIZER_MAP = {"BFGS": BFGS}
CALCULATOR_MAP = {"EMT": EMT(), "ALIGNN": alignn_calculator, "M3GNET": m3gnet_calculator}


def translate_to_bottom(structure):
    min_c = min(site.c for site in structure)
    translation_vector = [0, 0, -min_c + 0.1]  # 0.1 to ensure there's a small gap above z=0
    translated_structure = structure.copy()
    for site in translated_structure:
        site.coords += translation_vector
    return translated_structure


def create_interfaces(materials: Sequence[Structure], settings):
    """
    Create interfaces for the given materials and settings.
    """
    print("Creating interfaces...")
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["ZSL_PARAMETERS"]["MAX_AREA_TOL"],
        max_area=settings["ZSL_PARAMETERS"]["MAX_AREA"],
        max_length_tol=settings["ZSL_PARAMETERS"]["MAX_LENGTH_TOL"],
        max_angle_tol=settings["ZSL_PARAMETERS"]["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]],
        film_structure=materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]],
        substrate_miller=settings["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"],
        film_miller=settings["LAYER_PARAMETERS"]["MILLER_INDICES"],
        zslgen=zsl,
        strain_tol=settings["ZSL_PARAMETERS"]["STRAIN_TOL"],
    )

    # Find terminations
    cib._find_terminations()
    terminations = cib.terminations

    # Create interfaces for each termination
    interfaces = {}
    for termination in terminations:
        interfaces[termination] = []
        for interface in cib.get_interfaces(
                termination,
                gap=settings["INTERFACE_PARAMETERS"]["DISTANCE_Z"],
                film_thickness=settings["LAYER_PARAMETERS"]["THICKNESS"],
                substrate_thickness=settings["SUBSTRATE_PARAMETERS"]["THICKNESS"],
                in_layers=True,
        ):
            # Wrap atoms to unit cell
            interface["interface"].make_supercell((1, 1, 1), to_unit_cell=True)
            interfaces[termination].append(interface)
    return interfaces, terminations


def sort_interfaces(interfaces: dict, terminations: list, strain_mode="mean_abs_strain"):
    """
    Sort interfaces by the specified strain mode and number of sites.
    """
    sorted_interfaces = {}
    for termination in terminations:
        sorted_interfaces[termination] = sorted(
            interfaces[termination], key=lambda x: (x[strain_mode], x["interface"].num_sites)
        )
    return sorted_interfaces


def relax_structure(atoms: Atoms, optimizer: Optimizer, calculator: Calculator, fmax: float = 0.05,
                    verbose: bool = False):
    """
    Relax the structure using the specified optimizer and calculator.
    """
    # Set up the interface for relaxation
    ase_structure = atoms.copy()
    ase_structure.set_calculator(calculator)
    dyn = optimizer(ase_structure)
    dyn.run(fmax=fmax)

    # Extract results
    ase_original_structure = atoms
    ase_original_structure.set_calculator(calculator)
    ase_final_structure = ase_structure
    relaxed_energy = ase_structure.get_total_energy()

    # Print out the final relaxed structure and energy
    if verbose:
        print('Original structure:\n', ase_to_poscar(ase_original_structure))
        print('\nRelaxed structure:\n', ase_to_poscar(ase_final_structure))
        print(f"The final energy is {float(relaxed_energy):.3f} eV.")

    return ase_final_structure, relaxed_energy


def filter_atoms_by_tag(atoms: Atoms, material_index: int):
    """
    Filter ASE atoms by their tag, corresponding to the material index.
    """
    return atoms[atoms.get_tags() == material_index]


def get_material_visualization_html(material, title: str, rotation: str = '0x', number_of_repetitions: int = 3):
    """
    Returns an HTML string with a Base64-encoded image for visualization,
    including the name of the file, positioned horizontally.
    """
    # Set the number of unit cell repetition for the structure
    n = number_of_repetitions
    material_repeat = make_supercell(material, [[n, 0, 0], [0, n, 0], [0, 0, 1]])
    text = f"{material.symbols} - {title}"

    # Write image to a buffer to display in HTML
    buf = io.BytesIO()
    write(buf, material_repeat, format='png', rotation=rotation)
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    html_str = f'''
    <div style="display: inline-block; margin: 10px; vertical-align: top;">
        <p>{text}</p>
        <img src="data:image/png;base64,{img_str}" alt="{title}" />
    </div>
    '''
    return html_str


def create_relaxed_interface_with_min_strain(materials: Sequence[Structure], settings: dict, optimizer: Optimizer,
                                             calculator: Calculator):
    """
    Create an interface with minimal strain using ZSL and relaxes it using specified optimizer and calculator.
    """
    interfaces, terminations = create_interfaces(
        materials=materials,
        settings=settings
    )
    sorted_interfaces = sort_interfaces(interfaces, terminations)
    # TODO: run for every termination
    termination_index = 0
    interface_index = 0
    termination = terminations[termination_index]
    try:
        interface = sorted_interfaces[termination][interface_index]["interface"]
    except IndexError:
        raise ValueError("No interfaces found.")

    ase_original_interface = pymatgen_to_ase(interface)
    ase_final_interface, relaxed_energy = relax_structure(
        atoms=ase_original_interface,
        optimizer=optimizer,
        calculator=calculator,
        fmax=settings["RELAXATION_PARAMETERS"]["FMAX"],
        verbose=True
    )

    ase_substrate = filter_atoms_by_tag(ase_original_interface, settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"])
    ase_layer = filter_atoms_by_tag(ase_original_interface, settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"])
    ase_substrate_bulk = pymatgen_to_ase(materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]])
    ase_layer_bulk = pymatgen_to_ase(materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]])

    adhesion_energy = get_adhesion_energy(
        ase_final_interface,
        ase_substrate,
        ase_layer,
        calculator
    )

    interfacial_energy = get_interfacial_energy(
        ase_final_interface,
        ase_substrate,
        ase_substrate_bulk,
        ase_layer,
        ase_layer_bulk,
        calculator
    )

    html_original = get_material_visualization_html(ase_original_interface, "original", "-90x")
    html_relaxed = get_material_visualization_html(ase_final_interface, "relaxed", "-90x")

    # Display the interfaces before and after relaxation
    html_content = f'<div style="display: flex;">{html_original}{html_relaxed}</div>'
    display(HTML(html_content))

    return ase_final_interface, relaxed_energy, adhesion_energy, interfacial_energy

## 3. Import materials
From the `uploads` folder, import the materials in ESSE format to be used in the analysis.

In [ ]:
import os
import json
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

materials_in = []
current_folder = os.getcwd()
input_folder = "uploads"
for file in FILES:
    with open(f"{current_folder}/{input_folder}/{file}", "r") as f:
        data = f.read()
        materials_in.append(json.loads(data))

if "materials_in" in globals():
    pymatgen_materials = [to_pymatgen(item) for item in materials_in]
    if MILLER_INDICES["USE_CONVENTIONAL_CELL"]: pymatgen_materials = [
        SpacegroupAnalyzer(item).get_conventional_standard_structure() for
        item in pymatgen_materials]
    pymatgen_materials = [translate_to_bottom(material) for material in pymatgen_materials]
for material in pymatgen_materials:
    print(material, "\n")


## 4. Create relaxed interfaces with minimal energy among surfaces with different Miller indices
Cycle through HKL indices to create all matching interfaces with minimal strain

In [ ]:
from pymatgen.core.surface import SlabGenerator
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
# TODO: use Made package instead
from src.utils import ase_to_pymatgen
import hashlib

optimizer = OPTIMIZER_MAP[SETTINGS["RELAXATION_PARAMETERS"]["RELAXER"]]
calculator = CALCULATOR_MAP[SETTINGS["RELAXATION_PARAMETERS"]["CALCULATOR"]]


def get_structure_hash(structure: Atoms | Structure):
    """Get a hash for the structure."""
    if isinstance(structure, Atoms): structure = ase_to_pymatgen(structure)
    analyzer = SpacegroupAnalyzer(structure)
    standard_structure = analyzer.get_primitive_standard_structure()
    standard_structure_str = json.dumps(standard_structure.as_dict())
    return hashlib.md5(standard_structure_str.encode()).hexdigest()


results = []

if MILLER_INDICES["INCLUDE_NEGATIVE"]:
    MIN_MILLER = {key: -value for key, value in MILLER_INDICES["MAX_MILLER"].items()}
else:
    MIN_MILLER = {"H": 0, "K": 0, "L": 0}

original_search_area = SETTINGS["ZSL_PARAMETERS"]["MAX_AREA"]
MAX_RETRIES = 5  # Set a maximum number of retries to prevent infinite loops
AREA_INCREASE_FACTOR = 1.5  # The factor by which to increase the search area if no interfaces are found

for h in range(MIN_MILLER["H"], MILLER_INDICES["MAX_MILLER"]["H"] + 1):
    for k in range(MIN_MILLER["K"], MILLER_INDICES["MAX_MILLER"]["K"] + 1):
        for l in range(MIN_MILLER["L"], MILLER_INDICES["MAX_MILLER"]["L"] + 1):
            if h == 0 and k == 0 and l == 0:
                continue
            SETTINGS["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"] = (h, k, l)
            print(f"Creating interface for ({h},{k},{l})...")

            # Check if the equivalent interface for current Miller indices has already been created to skip heavy calculations
            substrate = pymatgen_materials[SETTINGS["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]]
            slabgen = SlabGenerator(substrate, (h, k, l), SETTINGS["SUBSTRATE_PARAMETERS"]["THICKNESS"], 10)
            substrate_surface = slabgen.get_slab(0, 10)
            substrate_hash = get_structure_hash(substrate_surface)

            existing_entry = next((entry for entry in results if entry["substrate_hash"] == substrate_hash), None)
            if existing_entry:
                existing_entry["miller_indices"].append((h, k, l))
                continue

            # Create the relaxed interface with retry if no interfaces are found for selected area
            retries = 0
            while retries < MAX_RETRIES:
                try:
                    interface, relaxed_energy, adhesion_energy, interface_energy = create_relaxed_interface_with_min_strain(
                        materials=pymatgen_materials,
                        settings=SETTINGS,
                        optimizer=optimizer,
                        calculator=calculator
                    )

                    print(f"Interfacial energy: {interface_energy:.3f} eV/Å^2 ({interface_energy / 0.16:.3f} J/m^2)")
                    results.append({"miller_indices": [(h, k, l)],
                                    "substrate_hash": substrate_hash,
                                    "metrics": {"adhesion_energy": adhesion_energy,
                                                "interfacial_energy": interface_energy},
                                    "structures": {"relaxed_interface": interface},
                                    "settings": SETTINGS
                                    })
                    break
                except ValueError as e:
                    if str(e) == "No interfaces found.":
                        print(
                            f"No interfaces found for ({h},{k},{l}). Increasing the area by {AREA_INCREASE_FACTOR} and retrying.")
                        SETTINGS["ZSL_PARAMETERS"]["MAX_AREA"] *= AREA_INCREASE_FACTOR
                        retries += 1
                    else:
                        raise

            SETTINGS["ZSL_PARAMETERS"]["MAX_AREA"] = original_search_area


## 5. Plot the results

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


# Function to format the Miller indices with a bar above negative numbers
def format_miller_index(h, k, l):
    return ''.join(f"{abs(index)}" + ('\u0305' if int(index) < 0 else '') for index in [h, k, l])


formula_sub = pymatgen_materials[SETTINGS["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]].composition.reduced_formula
formula_layer = pymatgen_materials[SETTINGS["LAYER_PARAMETERS"]["MATERIAL_INDEX"]].composition.reduced_formula

# We need to collect all equivalent Miller indices for each unique surface and plot them
x = []
y = []

for entry in results:
    # For each group of equivalent Miller indices, we add the interfacial energy to the y-values
    energy = entry["metrics"]["interfacial_energy"]
    for miller_index in entry["miller_indices"]:
        # Format each Miller index and add to x_labels
        formatted_index = format_miller_index(*miller_index)
        x.append(formatted_index)
        y.append(energy)

plt.bar(x, y)
plt.xlabel("Miller indices")
plt.ylabel("Interfacial energy (eV/Å^2)")
plt.title(f"Interfacial energy as a function of Miller indices. Substrate: {formula_sub}, 2D material: {formula_layer}")

fig = plt.gcf()
fig.set_size_inches(20, 5)
# plt.xticks(rotation=90)  # Rotate labels if they overlap
plt.show()



## 6. Return the most optimal interface

In [ ]:
# TODO: use Made package instead
from src.utils import calculate_average_interlayer_distance

# Find the most optimal interface
most_optimal_index = 5
most_optimal_miller_indices = x[most_optimal_index]
most_optimal_delta_energy = y[most_optimal_index]
ase_final_interface = results[most_optimal_index]["structures"]["relaxed_interface"]
print(
    f"The most optimal interface is ({most_optimal_miller_indices}) with the interfacial energy of {most_optimal_delta_energy:.3f} eV/Å^2 ({most_optimal_delta_energy / 0.16:.3f} J/m^2)")
print(f"The average interlayer distance: {calculate_average_interlayer_distance(ase_final_interface, 0, 1):.3f} Å.")

## 7. Visualize the interface

In [ ]:
from ase.visualize import view

view(ase_final_interface * [3, 3, 1])

## 8. Save the most optimal interface
Interface can be saved in POSCAR or ESSE format into the `uploads` folder.

In [ ]:
# TODO: use Made package instead
from src.utils import from_ase

poscar = ase_to_poscar(ase_final_interface)

# Add interface information to the POSCAR
interface_name = f"{formula_layer}-{formula_sub}({most_optimal_miller_indices}) interface"
poscar = f"{interface_name}\n" + poscar.split("\n", 1)[1]

print(f'POSCAR file written to `{input_folder}` folder:\n {poscar}')
with open(f"{input_folder}/{interface_name}.poscar", "w") as f:
    f.write(poscar)

# Uncomment below to save interface in ESSE format
# esse = from_ase(ase_final_interface)
# esse["metadata"]["interface_settings"] = SETTINGS
# with open(f"{input_folder}/{interface_name}.json", "w") as f:
#     f.write(json.dumps(esse))
